# Moment Tensor Inversion based on mtinvers (Dahm and Krüger, 2014)
### Adaption to python M. van Driel and S. Gessele 
 ![Yasur Photo: A. Gerst](./pictures/Yasur_Gerst.png)

In [ ]:
import matplotlib

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

import functools
import pathlib
import typing

from obspy import Trace

import numpy as np
from obspy.signal import util
from obspy.signal.filter import bandpass
from obspy.signal.invsim import cosine_taper
from obspy.imaging.mopad_wrapper import beachball
from obspy.imaging.scripts.mopad import MomentTensor
import time


In [ ]:
# functions that are used in function mtinv_amplitude_spec

#(back)transformation of 6 independent moment tensor components to set constraints in the inversion
#futher information: MT_transformation.pdf
def transform(M):
    m = np.zeros(6)
    m[0] = 0.5*(M[1] - M[0])
    m[1] = M[3]
    m[2] = M[4]
    m[3] = M[5]
    m[4] = (0.5*(M[0] + M[1]) - M[2])/3
    m[5] = (M[0] + M[1] + M[2])/3
    return m

def backtransform(m):
    M = np.zeros(6)
    M[0] = -m[0] + m[4] + m[5]
    M[1] = m[0] + m[4] + m[5]
    M[2] = m[5] - 2*m[4]
    M[3] = m[1]
    M[4] = m[2]
    M[5] = m[3]
    return M


In [ ]:
# function to calculate the error of the current MT model
# m = 6 independent components of MT model, gw = Greens tensor in freq. domain, 
#u = seismograms in freq. domain, nw = number of frequencies
# Further information: Tilt effects on moment tensor inversion in the near field of active volcanoes, 
#M. van Driel et al., 2015
# equation 6 (without weighting) is calculated for each freq.-> calculating L2 norm of all freq.
def error(m,gw,uw,nw):
    residual = np.zeros(nw)
    for w in range(nw):
        difference = abs(uw[:,w]) - abs(gw[:,:,w].dot(m).T)
        residual[w] = (difference.dot(difference.T) / abs(uw[:,w].T.dot(uw[:,w])))[0][0]
    residual /= nw
    return ((residual).sum())**0.5

def error_all(m,gw,uw,nw):
    residual = np.zeros(nw)
    for w in range(nw):
        difference = abs(uw[:,w] - gw[:,:,w].dot(m[:,w]).T)
        residual[w] = (difference.dot(difference.T) / abs(uw[:,w].T.dot(uw[:,w])))
    residual /= nw
    return (residual.sum())**0.5


In [ ]:
# computing seismograms from best solution

def compute_seismograms(st_G,m,s_t,station_name,rot,fmax,fmax_hardcut_factor):
    df = st_G[0].stats.sampling_rate
    dt = st_G[0].stats.delta
           
    nG = st_G[0].stats.npts
    nfft = util.next_pow_2(nG * 2)
    nfinv = fmax_hardcut_factor * fmax / (df / 2.) * nfft 
    nw = min(nfft/2+1,nfinv)
    
    if rot:
        cha_ = ["HHX","HHY","HHZ","HJX","HJY","HJZ"]
        compo = 6
    else:
        cha_ = ["HHX","HHY","HHZ"]
        compo = 3
        
    Gw = np.zeros((compo,6, int(nfft/2 + 1))) * 0j
    Ug = np.zeros((compo,int(nfft/2 + 1))) * 0j
        
    for _i,cha in enumerate(cha_):      
        for _j in np.arange(6):
            Gval = st_G.select(station='%s%d' %(station_name,_j), channel=cha)[0].data
            Gw[_i,_j,:] = np.fft.rfft(Gval, n=nfft) 
            
    if type(s_t).__name__=='float':
        s_f = 1. 
        for w in range(int(nw)):
            Ug[:,w] = Gw[:,:,w].dot(m.T)
    else:
        s_f = np.fft.rfft(s_t, n=nfft) 
        for w in range(int(nw)):
            Ug[:,w] = Gw[:,:,w].dot(m*s_f[w]).T
    inv_u = Stream()

#creating obspy traces ....
    for j,_c in enumerate(cha_):
        tr_u = Trace()
        tr_u.stats.network = "XX"
        tr_u.stats.station = station_name
        tr_u.stats.sampling_rate = df
        tr_u.stats.ndata = nG
        sg = np.fft.irfft(Ug[j,:],n=nfft) 
        tr_u.data = sg[0:nG]
        tr_u.stats.channel = _c
        inv_u.append(tr_u)
    return inv_u
            

In [ ]:
def mtinv(receiver_l,st_u, st_G, fmax, fmax_hardcut_factor=8, S0=1., nsv=1,\
          single_force=False, stat_subset=[], force_recalc=False, weighting=False,\
          constraint=0,rot=False,varn=0.01):

    '''
    Function provides full waveform method using amplitude spectra for moment tensor inversion.
    Further information: mtinvers_manual.pdf chapter 5
    Returns numpy.array of six MT components

    :type st_u: obspy Stream object
    :param st_u: Stream containing the seismograms, station names are numbers
        starting with 0, channels are of format '%1d' where the
        number is the component of the receiver ([0,1,2])
    :type st_G: obspy Stream object
    :param st_G: Stream containing the green's functions, station names are numbers
        starting with 0, channels are of format '%1d%1d' where the first
        number is the component of the receiver ([0,1,2]) and the second the
        source ([0,1,...,5]) (Due to the constraint implementation this should be
        arranged like xx,yy,zz,xy,xz,yz. See MT_transformation.pdf). Should have the same sample rate as st_tr.
    :type fmax: float 
    :param fmax: maximum frequency which is inverted for  - needed if fmax_hardcut_factor is set
    :type fmax_hardcut_factor: float 
    :param fmax_hardcut_factor: only lower parts of the frequency spectrum is inverted for - much faster but
        less precise
    :type S0: numpy.ndarray or float
    :param S0: source time function used for comupation of the greensfunction
        to deconvolve. Needs to have same sample rate as st_tr
    :type nsv: int
    :param nsv: how many solutions are expected 
    
    :type constraint: int
    :param constraint: constraints for the moment tensor inversion: 0 = full MT, 1 = without volume change,
        2 = im-/explosion
    :type rot: bool
    :param rot: if true also 3 C of rotational data and rotational greens functions are expected 
    '''

    df = st_u[0].stats.sampling_rate
    dt = st_u[0].stats.delta

    max_u = np.max(np.abs(st_u.max()))
    min_r = np.min(np.abs(st_u.max()))

    stat_code = receiver_l
    #print(stat_code)
    nstat = len(stat_code)
    ndat = st_u[0].data.size
    nG = st_G[0].data.size
    nfft = util.next_pow_2(max(ndat, nG) * 2)

    # going to frequency domain
    # correction for stf in green's forward simulation
    if type(S0).__name__=='float':
        S0w = 1.
    else:
        S0w = np.fft.rfft(S0, n=nfft) #* dt


    # setup seismogram matrix in fourier space (uw)
    if rot:
        compo = 6
    else:
        compo = 3

    u = np.zeros((nstat * compo, ndat))
    uw = np.zeros((nstat * compo, int(nfft/2+1))) * 0j
    weightsr = np.zeros(nstat * compo)

    #adding noise to synthetics...
    noise = np.random.randn(nstat*compo,ndat)
    var = varn*max_u
    var_rot = varn*min_r

    for k,station in enumerate(stat_code):
        u[k*compo,:] = (st_u.select(station=station, channel='HHX')[0].data+var*noise[k*compo,:])
        u[k*compo + 1,:] = (st_u.select(station=station, channel='HHY')[0].data+var*noise[k*compo+1,:])
        u[k*compo + 2,:] = (st_u.select(station=station, channel='HHZ')[0].data+var*noise[k*compo+2,:])
        uw[k*compo,:] = np.fft.rfft(u[k*compo,:], n=nfft) #* dt 
        uw[k*compo+1,:] = np.fft.rfft(u[k*compo+1,:], n=nfft) #* dt 
        uw[k*compo+2,:] = np.fft.rfft(u[k*compo+2,:], n=nfft) #* dt 

        if rot:
            u[k*compo + 3,:] = (st_u.select(station=station, channel='HJX')[0].data+var_rot*noise[k*compo+3,:])
            u[k*compo + 4,:] = (st_u.select(station=station, channel='HJY')[0].data+var_rot*noise[k*compo+4,:])
            u[k*compo + 5,:] = (st_u.select(station=station, channel='HJZ')[0].data+var_rot*noise[k*compo+5,:])
            uw[k*compo+3,:] = np.fft.rfft(u[k*compo+3,:], n=nfft) #* dt
            uw[k*compo+4,:] = np.fft.rfft(u[k*compo+4,:], n=nfft) #* dt
            uw[k*compo+5,:] = np.fft.rfft(u[k*compo+5,:], n=nfft) #* dt

        for i in range(compo):
            weightsr[k*compo + i] = 1./(u[k*compo + i,:]**2).sum()**.5

    # setup greens matrix in fourier space (Gw)
    G = np.zeros((nstat * compo, 6, nG))
  #  print(G.shape)
    Gw = np.zeros((nstat * compo, 6, int(nfft/2+1))) * 0j

    Gval = np.zeros([6,nG]) #helper-variable to store values correctly for transformation
    con = np.ones(6) #array to set parts of G to zero wrt MT constraints (if constraint == 0  -> full MT)
    #MT without volume change
    if constraint == 1:
        con[5] = 0
    #im-/explosion
    elif constraint == 2:
        con[0:5] = 0.

    if rot:
        cha_ = ["HHX","HHY","HHZ","HJX","HJY","HJZ"]
    else:
        cha_ = ["HHX","HHY","HHZ"]
   
    #loop over all stations (k),seismogram components (i), G components(j) and implement constraints in G
    for k,station in enumerate(stat_code):
        for i,cha in enumerate(cha_):
            for j in np.arange(6):
                Gval[j] = st_G.select(station='%s%d' %(station,j), channel=cha)[0].data
                #print(st_G.select(station='%s%d' %(station,j), channel=cha))

            #transformation of G components
            G[k*compo + i,0,:] = con[0]*(Gval[1,:] - Gval[0,:])
            G[k*compo + i,1,:] = con[1]*Gval[3,:]
            G[k*compo + i,2,:] = con[2]*Gval[4,:]
            G[k*compo + i,3,:] = con[3]*Gval[5,:]
            G[k*compo + i,4,:] = con[4]*(Gval[0,:] + Gval[1,:] - Gval[2,:] * 2)
            G[k*compo + i,5,:] = con[5]*(Gval[0,:] + Gval[1,:] + Gval[2,:])
            for j in np.arange(6):
                Gw[k*compo + i,j,:] = np.fft.rfft(G[k*compo + i,j,:], n=nfft)/S0w # * dt / S0w deconvolving the stf from greensf


    # inversion
    # setup channel subset from station subset
    if stat_subset == []:
        stat_subset = np.arange(nstat)
    else:
        stat_subset = np.array(stat_subset)

    subsi = np.zeros(np.array(stat_subset).size*3, dtype=int)
    for i in np.arange(np.array(stat_subset).size):
        subsi[i*3:(i+1)*3] = stat_subset[i]*3 + np.array([0,1,2])
    stat_subset = subsi

    # setup weights matrix
    weightsr /= weightsr.min()
    #print('weights = ' + str(weightsr))
    weightsr = np.matrix(np.diag(weightsr[stat_subset]))

    M = np.zeros((6 + single_force * 3, int(nfft/2+1))) * 0j

    # use frequencies below corner frequency * fmax_hardcut_factor (tremendous speed up in inversion)
    nfinv = fmax_hardcut_factor * fmax / (df / 2.) * nfft

    # use all frequencies (theoretically more exact, but slower)
    #nfinv = nfft/2+1                   
    nfinv = min(nfft/2+1, nfinv)
    
    for w in np.arange(int(nfinv)):
        if weighting:
            Gg = weightsr * np.matrix(Gw[[stat_subset],:,w])
        else:
            Gg = np.matrix(Gw[[stat_subset],:,w])
        GI = np.linalg.pinv(Gg, rcond=0.00001)

        if weighting:
            m = GI * weightsr * np.matrix(uw[[stat_subset],w]).T
        else:
            m = GI * np.matrix(uw[[stat_subset],w]).T

        M[:,w] = np.array(m.flatten())

    err =  error_all(M,Gw,uw,int(nfinv))

    # back to time domain
    M_t = np.zeros((6 + single_force * 3, ndat))

    for j in np.arange(6 + single_force * 3):
        M_t[j,:] = np.fft.irfft(M[j,:],n=ndat) #[:nfft] * df
        # detrend
        x1, x2 = M_t[j,0], M_t[j,-1]
        ndat = M_t[j,:].size
        M_t[j,:] -= (x1 + np.arange(ndat) * (x2 - x1) / float(ndat - 1))
        # taper
        M_t[j,:] *= cosine_taper(ndat, 0.01)

    # Filter
    M_t = bandpass(M_t, fmin, fmax, df)
    # use principal component analysis for constrained inversion

    U, s, V = np.linalg.svd(M_t, full_matrices=False)
    m = []
    x = []
    for k in np.arange(nsv):
        m.append(np.matrix(U[:,k]).T)
        x.append(np.matrix(V[k] * s[k]))

    argm = np.abs(m[0]).argmax()
    sig = np.sign(m[0][argm,0])
    M_t = np.zeros((M_t.shape))
    Mret = []
    for k in np.arange(nsv):
        m[k] *= sig
        x[k] *= sig
        M_t += np.array(m[k] * x[k])
        Mret.append(backtransform(np.array(m)[0]))

    x = np.array(x).flatten()
    
    return Mret, s, x, err


In [ ]:
##############################################################################
# End of definition block
# Main program
##############################################################################
from obspy import *
from obspy.core import read,UTCDateTime
from numpy.random import rand,uniform
# read streams for Greens functions and seismograms (used Convert2Stream)
stf = amplitude = 1.
rot = True
var_noise = 0.000001
fmin=0.8
fmax= 8
fmax_hard= 1
S0 = stf #1.
weight = True
varn = var_noise
stat_subset=[]
constraint = 0

receiver_name = ["SCIAR","SROC","HELI","PORT","RINA","LISC"]

event_depth = 200 #600
#### Reading the pre-computed Greensfunctions
st_G = Stream()
for receiver in (receiver_name): 
    for jj in range(6):
            st_db = read("./Stromb_2022_GF/%s_GF%d_%d0?.sac"%(receiver,jj+1,event_depth))
            tr = st_db.select(channel="HHX")[0]
            tr.stats.station = "%s%d"%(receiver,jj)
            st_G.append(tr)
            tr = st_db.select(channel="HHY")[0]
            tr.stats.station = "%s%d"%(receiver,jj)
            st_G.append(tr)
            tr = st_db.select(channel="HHZ")[0]
            tr.stats.station ="%s%d"%(receiver,jj)
            st_G.append(tr)
            
            if rot == True:
                tr = st_db.select(channel="HJX")[0]
                tr.stats.station = "%s%d"%(receiver,jj)
                st_G.append(tr)
                tr = st_db.select(channel="HJY")[0]
                tr.stats.station = "%s%d"%(receiver,jj)
                st_G.append(tr)
                tr = st_db.select(channel="HJZ")[0]
                tr.stats.station = "%s%d"%(receiver,jj)
                st_G.append(tr)            

# reading the data
st_u = Stream()
for receiver in receiver_name:
        dummy = read('./hom_90-45Crack/%s_hom-90-45crack_%d??.sac'%(receiver,event_depth))
        #dummy = read('./2L_90-45Crack/%s_Stromboli_2L_90-45Crack_%d_400??.sac'%(receiver,event_depth))
        if rot == True:
            st_u.append(dummy.select(channel="HHX")[0])
            st_u.append(dummy.select(channel="HHY")[0])
            st_u.append(dummy.select(channel="HHZ")[0])
            st_u.append(dummy.select(channel="HJX")[0])
            st_u.append(dummy.select(channel="HJY")[0])
            st_u.append(dummy.select(channel="HJZ")[0])
            compo = 6
        else:
            st_u.append(dummy.select(channel="HHX")[0])
            st_u.append(dummy.select(channel="HHY")[0])
            st_u.append(dummy.select(channel="HHZ")[0])
            compo = 3

#st_G.differentiate() # for synthetic data we are in the same space
st_G.filter("bandpass",freqmin=fmin,freqmax=fmax)
st_u.taper(type="cosine",max_percentage=0.1)
st_u.filter("bandpass",freqmin=fmin,freqmax=fmax)

# resampling to common data 
df = fmax * 10.
st_u.resample(sampling_rate=df)
st_G.resample(sampling_rate=df)
#stf_tr.resample(sampling_rate=df)

# Now the inversion ....
MTr,ew,so,errorm =  mtinv(receiver_name,st_u, st_G,fmax, fmax_hardcut_factor=1, S0=amplitude, nsv=1,\
      single_force=False, stat_subset=stat_subset, force_recalc=False, weighting=True,constraint=constraint,\
                      rot=rot,varn=var_noise)
scale = (np.max(so) - np.min(so))
so /= np.max(np.abs(so))
#if nsv > 1 we have to change it here
MT = MTr[0] * ew[0]/2#*scale
MT_out = MomentTensor(MT, system='XYZ')
print('iso percentage = %02d' % MT_out.get_iso_percentage())
print('DC percentage = %02d' % MT_out.get_DC_percentage())
print('CLVD percentage = %02d' % MT_out.get_CLVD_percentage())

print('Minimum Error = %r\n'%errorm)
print('MT Tensor: ',MT)

beachball(MT,mopad_basis='XYZ')


#now  plotting the traces ...
for _j, rec in enumerate(receiver_name):

    st_us = st_u.select(station=rec)
    st_ug = compute_seismograms(st_G,MT,1.,rec,rot,fmax,1)
    
    plt.figure(figsize=(8, 16))
    for _k, (tr_db, tr_gt) in enumerate(zip(st_ug, st_us)):
        plt.subplot(len(st_ug), 1, _k + 1)
        plt.title("%s"%(tr_db.id))
        plt.plot(tr_gt.data, label="Ground truth")
        plt.plot(tr_db.data, label="DB")
    plt.legend()
    plt.show()

    #plt.savefig('%s-%s_%s_%d.png'%("MTI-crack-sub-2",rot,receiver_name[_j],_i))
    plt.close()
